In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "purchase_data.json"
itemPurch = pd.read_json(file)

normPurch = itemPurch.groupby("SN")
itemPurch.head(10)



,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [3]:
# BASIC SUMMARY
# number of available items
items = itemPurch["Item ID"].unique()
numItems = len(items)

# average price of each purchase
avgPurch = itemPurch["Price"].mean()

# number of recorded purchases
numPurch = len(itemPurch)

#total revenue of recorded purchases
totalRev = itemPurch["Price"].sum()

#generate df of results
purchAnalysis = pd.DataFrame({"Number of Items":[numItems],
                              "Average Purchase Price":[avgPurch],
                              "Number of Purchases":[numPurch],
                              "Total Revenue":[totalRev]})
purchAnalysis = purchAnalysis.reindex(columns=["Number of Items","Average Purchase Price","Number of Purchases","Total Revenue"])

# format avg. purchase and total revenue as USD$
purchAnalysis["Average Purchase Price"]= ['${0:.2f}'.format(val) for val in purchAnalysis["Average Purchase Price"]]
purchAnalysis["Total Revenue"]= ['${0:.2f}'.format(val) for val in purchAnalysis["Total Revenue"]]

purchAnalysis

,Number of Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,$2.93,780,$2286.33


In [4]:
# DEMOGRAPHIC ANALYSIS BY GENDER GROUP
# remove extra entries info from users who logged more than one purchase
uniqueSN = itemPurch.drop_duplicates(subset=["SN"])

# group the DF by gender to find count of users per gender
byGender = uniqueSN.groupby("Gender")
genders = pd.DataFrame(byGender["SN"].count())
genders = genders.rename(columns = {"SN":"Count of Players"})

# calculate each groups' percentage of total playerbase
totalPlyrs = genders["Count of Players"].sum()
genders["% of Players"] = genders["Count of Players"]/totalPlyrs
genders["% of Players"] = ['{0:.2f}%'.format(val*100) for val in genders["% of Players"]]

genders


,Count of Players,% of Players
Gender,,
Female,100,17.45%
Male,465,81.15%
Other / Non-Disclosed,8,1.40%


In [5]:
# PURCHASING ANALYSIS BY GENDER GROUP
# group DF of purchases by gender
genBuys = itemPurch.groupby("Gender")

# calculate mean purchase price by group and format as USD$
genPurch = pd.DataFrame(genBuys["Price"].mean())
genPurch["Price"] = ['${0:.2f}'.format(val) for val in genPurch["Price"]]

# calculate total revenue by group and format as USD$
genPurch["Total Revenue"] = genBuys["Price"].sum()
genPurch["Total Revenue"] = ['${0:.2f}'.format(val) for val in genPurch["Total Revenue"]]

# calculate number of purchases by group
genPurch["Number of Purchases"] = genBuys["Price"].count()
genPurch = genPurch.rename(columns={"Price":"Average Purchase"})

# to normalize, group by Gender and SN
uniqueSNs = itemPurch.groupby(["Gender","SN"])
# normPurch finds mean purchase price for each SN, while holding the Gender of each SN
normPurch = pd.DataFrame(uniqueSNs["Price"].mean())
# unstack and find the mean of each row (Male, Female, Nonspecific)
normPurch = normPurch.unstack()
normTotals = pd.DataFrame(normPurch.mean(axis=1))

# format normalized averages to USD$ and join with summary table
genPurch = genPurch.join(normTotals)
genPurch[0] = ['${0:.2f}'.format(val) for val in genPurch[0]]
genPurch = genPurch.rename(columns={0:"Normalized Average Purchase"})

genPurch


,Average Purchase,Total Revenue,Number of Purchases,Normalized Average Purchase
Gender,,,,
Female,$2.82,$382.91,136,$2.81
Male,$2.95,$1867.68,633,$2.95
Other / Non-Disclosed,$3.25,$35.74,11,$3.34


In [6]:
# DEMOGRAPHIC ANALYSIS BY AGE
# create bins to group player age by
# use uniqueSN df, no repeat entries of players
ageBins = np.arange(9,50,5)
ageBins = np.insert(ageBins,0,int(0))
ageBins

#create labels for the bins
binLabels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49"]

#group the df by age range
byAge = pd.cut(uniqueSN["Age"],ageBins,labels=binLabels)

byAge = pd.DataFrame(byAge.value_counts())
byAge = byAge.reindex(binLabels)
byAge = byAge.rename(columns={"Age":"Number of Players"})

# calculate percentage of total players in each age range, format as %
byAge["% of Total"] = byAge["Number of Players"] / (len(uniqueSN["SN"])) 
byAge["% of Total"] = ['{0:.2f}%'.format(val*100) for val in byAge["% of Total"]]
byAge

,Number of Players,% of Total
<10,19,3.32%
10-14,23,4.01%
15-19,100,17.45%
20-24,259,45.20%
25-29,87,15.18%
30-34,47,8.20%
35-39,27,4.71%
40-44,10,1.75%
45-49,1,0.17%


In [7]:
# PURCHASE ANALYSIS BY AGE GROUP
# establish bins to analyze purchase data by age group
ageBins = np.arange(9,50,5)
ageBins = np.insert(ageBins,0,int(0))
binLabels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49"]

# create new column in df by binning age into ranges
itemPurch["Age Group"] = pd.cut(itemPurch["Age"],ageBins,labels=binLabels)
buyAge = itemPurch.groupby("Age Group")

# create 3 dataframes that hold purchase info for each age group
purchCount = pd.DataFrame(buyAge["SN"].count()).rename(columns={"SN":"Number of Purchases"})
avgPrice = pd.DataFrame(buyAge["Price"].mean()).rename(columns={"Price":"Average Purchase Value"})
ageTotal = pd.DataFrame(buyAge["Price"].sum()).rename(columns={"Price":"Total Revenue"})

# join the dataframes on their index, "Age Group" (default)
ageMerge = purchCount.join(avgPrice,how='outer')
ageMerge = ageMerge.join(ageTotal,how='outer')

# to normalize the average purchases and remove redundancy, group by age group and screenname
uniqueAges = itemPurch.groupby(["Age Group","SN"])
# make a dataframe of the average purchase per SN
normAge = pd.DataFrame(uniqueAges["Price"].mean())

# unstack and take the sum of each row to find gender group averages, normalized
normAge = normAge.unstack()
normAge = pd.DataFrame(normAge.mean(axis=1))

# merge with ageMerge, format as USD$ and relabel column
ageMerge = ageMerge.join(normAge)
ageMerge[0] = ['${0:.2f}'.format(val) for val in ageMerge[0]]
ageMerge = ageMerge.rename(columns={0:"Normalized Average Purchase"})

# format average spending and total revenue to USD$
ageMerge["Average Purchase Value"] = ['${0:.2f}'.format(val) for val in ageMerge["Average Purchase Value"]]
ageMerge["Total Revenue"] = ['${0:.2f}'.format(val) for val in ageMerge["Total Revenue"]]
ageMerge.fillna("0",inplace=True)

ageMerge


,Number of Purchases,Average Purchase Value,Total Revenue,Normalized Average Purchase
Age Group,,,,
<10,28,$2.98,$83.46,$3.00
10-14,35,$2.77,$96.95,$2.75
15-19,133,$2.91,$386.42,$2.89
20-24,336,$2.91,$978.77,$2.90
25-29,125,$2.96,$370.33,$3.01
30-34,64,$3.08,$197.25,$3.16
35-39,42,$2.84,$119.40,$2.80
40-44,16,$3.19,$51.03,$3.22
45-49,1,$2.72,$2.72,$2.72


In [8]:
# BIGGEST SPENDERS IN PYMOLI
# group purchase df by screenname (SN) to see who bought the most
highRollers = itemPurch.groupby("SN")

# create df holding the 5 largest spenders
bigMoney = pd.DataFrame(highRollers["Price"].sum().sort_values(ascending=False).head(5))
bigMoney = bigMoney.rename(columns={"Price":"Total Spending"})

# create df holding most frequent buyers
freqBuyers = pd.DataFrame(highRollers["Price"].count().sort_values(ascending=False))
freqBuyers = freqBuyers.rename(columns={"Price":"Number of Purchases"})

# merge the two dataframes and add a column for average purchase cost
bigMoney = bigMoney.join(freqBuyers)
bigMoney["Average Purchase Value"] = bigMoney["Total Spending"]/bigMoney["Number of Purchases"]

# format to USD$ and set column index
bigMoney["Total Spending"] = ['${0:.2f}'.format(val) for val in bigMoney["Total Spending"]]
bigMoney["Average Purchase Value"] = ['${0:.2f}'.format(val) for val in bigMoney["Average Purchase Value"]]
bigMoney = bigMoney.reindex(columns = [["Number of Purchases","Average Purchase Value","Total Spending"]])

bigMoney


,Number of Purchases,Average Purchase Value,Total Spending
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [9]:
#MOST POPULAR ITEMS
# group by Item ID - ("Item Name" can have multiple item IDs and prices assigned to it, so it's no good to use)
itemPerformance = itemPurch.groupby("Item ID")

# create a dataframe storing the item IDs that appear most frequently
armory = pd.DataFrame(itemPerformance["Item ID"].count().sort_values(ascending=False).head(5))
armory = armory.rename(columns={"Item ID":"Number of Purchases"})

# create a second dataframe that holds total revenue for each item ID
armoryRev = pd.DataFrame(itemPerformance["Price"].sum().sort_values(ascending=False))
armoryRev = armoryRev.rename(columns={"Price":"Total Revenue"})

# join the two dfs, add a column for unit price, format to USD$
armoryFull = armory.join(armoryRev)
armoryFull["Unit Price"] = armoryFull["Total Revenue"]/armoryFull["Number of Purchases"]
armoryFull["Total Revenue"] = ['${0:.2f}'.format(val) for val in armoryFull["Total Revenue"]]
armoryFull["Unit Price"] = ['${0:.2f}'.format(val) for val in armoryFull["Unit Price"]]

uniqueItems = itemPurch.drop_duplicates(subset=["Item ID"])
uniqueItems = uniqueItems[["Item ID","Item Name"]]
uniqueItems.set_index("Item ID",inplace=True)

armoryFull = armoryFull.join(uniqueItems)
armoryFull = armoryFull[["Item Name","Unit Price","Number of Purchases","Total Revenue"]]

armoryFull = armoryFull.set_index([armoryFull.index,"Item Name"])
armoryFull



,,Unit Price,Number of Purchases,Total Revenue
Item ID,Item Name,,,
84,Arcane Gem,$2.23,11,$24.53
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
31,Trickster,$2.07,9,$18.63
34,Retribution Axe,$4.14,9,$37.26
175,Woeful Adamantite Claymore,$1.24,9,$11.16


In [10]:
# MOST PROFITABLE ITEMS
# group df by item ID and sum the price column to receive total revenue per item ID
# sort in order of total revenue, descending
itemPerformance = itemPurch.groupby("Item ID")
armoryRev = pd.DataFrame(itemPerformance["Price"].sum().sort_values(ascending=False)).head()
armoryRev = armoryRev.rename(columns={"Price":"Total Revenue"})

# create dataframe for the count of purchases per item ID
armoryCount = pd.DataFrame(itemPerformance["Item ID"].count())
armoryCount = armoryCount.rename(columns={"Item ID":"Purchase Count"})

# join into output table
armoryMaxRev = armoryRev.join(armoryCount)
armoryMaxRev = armoryMaxRev.join(uniqueItems)

# calculate unit price by dividing revenue/purchase count
armoryMaxRev["Unit Price"] = armoryMaxRev["Total Revenue"]/armoryMaxRev["Purchase Count"]

# add row index of Item Name and set column index
armoryMaxRev = armoryMaxRev.set_index([armoryMaxRev.index,"Item Name"])
armoryMaxRev = armoryMaxRev[["Total Revenue","Unit Price","Purchase Count"]]
# format unit price and total revenue to USD$
armoryMaxRev["Unit Price"] = ['${0:.2f}'.format(val) for val in armoryMaxRev["Unit Price"]]
armoryMaxRev["Total Revenue"] = ['${0:.2f}'.format(val) for val in armoryMaxRev["Total Revenue"]]

armoryMaxRev


,,Total Revenue,Unit Price,Purchase Count
Item ID,Item Name,,,
34,Retribution Axe,$37.26,$4.14,9
115,Spectral Diamond Doomblade,$29.75,$4.25,7
32,Orenmir,$29.70,$4.95,6
103,Singed Scalpel,$29.22,$4.87,6
107,"Splitter, Foe Of Subtlety",$28.88,$3.61,8
